<a target="_blank" href="https://colab.research.google.com/github/michalis0/Cloud-and-Advanced-Analytics/blob/main/labs/week_02/solutions/week_02_exercises_big_query_sol.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Google BigQuery

## Walkthrough

### Authentication

In [1]:
import os

PROJECT_NAME = "cloud-analytics-init" # REPLACE WITH YOUR PROJECT NAME
PATH_TO_KEY = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "google_key.json") # REPLACE WITH YOUR PATH TO KEY
PATH_TO_KEY

'/Users/etiennebruno/Library/CloudStorage/OneDrive-epfl.ch/ETIENNE/EPFL/EPFL_Master/MA4_EPFL/TAs/CAA/google_key.json'

#### For Colab users (recommended)

In [2]:
# from google.colab import auth
# auth.authenticate_user()
# print("Authenticated")

#### For Jupyter users ONLY (more challenging)

Follow [this](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) guide to create the JSON with the service account key. Then, make sure to replace "PATH_TO_CREDENTIALS_FILE" with the *absolute* path to the JSON service account key (e.g., "C:/Users/John/credentials.json"). 

In [3]:
%pip install google-cloud-bigquery
%pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = PATH_TO_KEY

### Connect to BigQuery

In [5]:
import pandas as pd
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project=PROJECT_NAME)

To run the cells below, you must first create a new dataset in BigQuery and upload the data from week 2 (`Teams.csv` and `Players.csv`) as tables. Since the CSV files are well-formed, you can automatically infer the schema when loading the data.

In [6]:
# Create a reference to the World Cup dataset
dataset_ref = client.dataset("world_cup", project=PROJECT_NAME)  # CHANGE WITH YOUR OWN DATASET (NOT TABLE) NAME AND PROJECT ID

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [7]:
# List the tables in the dataset
tables = list(client.list_tables(dataset))
for table in tables:  
    print(table.table_id)

players
players_and_teams
teams


In [8]:
# Create a reference to the "Teams" table and fetch the table
table_ref = dataset_ref.table("teams") # Make sure it matches the table name printed above (case sensitive)
teams_table = client.get_table(table_ref)

# Same for the "Players" table
table_ref = dataset_ref.table("players") # Make sure it matches the table name printed above (case sensitive)
players_table = client.get_table(table_ref)

In [9]:
# Display the schema of the "Teams" table 
players_table.schema

[SchemaField('surname', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('team', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('position', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('minutes', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('shots', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('passes', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('tackles', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('saves', 'INTEGER', 'NULLABLE', None, None, (), None)]

In [10]:
# Show a preview of the "Teams" table using pandas
client.list_rows(teams_table, max_results=5).to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


### Example of queries

In [11]:
q1 = """
select *
from %s.world_cup.teams
where ranking < 20
order by ranking
""" %PROJECT_NAME

query_job = client.query(q1)
query_job.to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Brazil,1,5,3,1,1,9,4,7,2
1,Spain,2,6,5,0,1,7,2,3,0
2,Portugal,3,4,1,2,1,7,1,8,1
3,Netherlands,4,6,6,0,0,12,5,15,0
4,Italy,5,3,0,2,1,4,5,5,0
5,Germany,6,6,4,0,2,13,3,8,1
6,Argentina,7,5,4,0,1,10,6,7,0
7,England,8,4,1,2,1,3,5,6,0
8,France,9,3,0,1,2,1,4,6,1
9,Greece,13,3,1,0,2,2,5,5,0


In [12]:
# Join the two tables
q2 = """
select teams.*, players.surname, players.position, players.shots,
players.minutes, players.passes, players.tackles, players.saves
from %s.world_cup.teams teams
join %s.world_cup.players players
on teams.team = players.team
""" %(PROJECT_NAME, PROJECT_NAME)

query_job = client.query(q2)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,surname,position,shots,minutes,passes,tackles,saves
0,Honduras,38,3,0,1,2,0,3,7,0,Jerry Palacios,forward,0,84,19,0,0
1,Honduras,38,3,0,1,2,0,3,7,0,Walter Martinez,forward,0,125,43,1,0
2,Honduras,38,3,0,1,2,0,3,7,0,Chavez,defender,0,270,62,3,0
3,Honduras,38,3,0,1,2,0,3,7,0,Izaguirre,defender,0,180,51,8,0
4,Honduras,38,3,0,1,2,0,3,7,0,Mauricio Sabilluen,defender,0,90,38,5,0
5,Honduras,38,3,0,1,2,0,3,7,0,Mendoza,defender,0,180,51,3,0
6,Honduras,38,3,0,1,2,0,3,7,0,Valladares,goalkeeper,0,270,51,0,12
7,Honduras,38,3,0,1,2,0,3,7,0,Espinoza,midfielder,0,135,36,4,0
8,Honduras,38,3,0,1,2,0,3,7,0,Guevara,midfielder,0,156,60,6,0
9,Honduras,38,3,0,1,2,0,3,7,0,Thomas,midfielder,0,114,37,5,0


### Basic Queries

In [13]:
query = "SELECT * FROM {}.world_cup.players LIMIT 5".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team,position,minutes,shots,passes,tackles,saves
0,Milito,Argentina,forward,91,0,33,0,0
1,Grafite,Brazil,forward,5,0,4,0,0
2,Kiessling,Germany,forward,7,0,5,0,0
3,Adiyiah,Ghana,forward,33,0,9,0,0
4,Amoah,Ghana,forward,11,0,4,1,0


In [14]:
query = "SELECT * FROM {}.world_cup.teams LIMIT 5".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


In [15]:
query = """SELECT surname, team, minutes, passes 
            FROM {}.world_cup.players 
            WHERE team="Switzerland" AND surname LIKE "%er%" 
            ORDER BY surname""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team,minutes,passes
0,Derdiyok,Switzerland,191,47
1,Fernandes,Switzerland,212,56
2,Inler,Switzerland,270,138
3,Lichtsteiner,Switzerland,270,80
4,Senderos,Switzerland,36,9
5,Ziegler,Switzerland,270,71
6,von Bergen,Switzerland,234,79


In [16]:
query = """SELECT team, COUNT(*) AS number_players_listed
            FROM {}.world_cup.players 
            GROUP BY team
            HAVING number_players_listed<20
            ORDER BY number_players_listed DESC
            LIMIT 5""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,number_players_listed
0,Greece,19
1,Brazil,19
2,Germany,19
3,Honduras,19
4,Ghana,19


In [17]:
query = """SELECT surname, position, minutes, T.team, ranking, games, wins, draws, losses
            FROM {}.world_cup.players as P
            JOIN {}.world_cup.teams AS T
            ON P.team=T.team
            WHERE surname <"C"
            LIMIT 5""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,position,minutes,team,ranking,games,wins,draws,losses
0,Adiyiah,forward,33,Ghana,32,5,2,2,1
1,Amoah,forward,11,Ghana,32,5,2,2,1
2,Bautista,forward,45,Mexico,17,4,1,1,2
3,Bunjaku,forward,13,Switzerland,24,3,1,1,1
4,Abreu,forward,72,Uruguay,16,6,3,2,1


## Exercise 1 - World Cup datasets

### Simple Queries
The first three questions are already solved for you, so that you have concrete examples of queries. Try to solve the remaining ones!

*1)  Which player on a team ending with "ia" played less than 200 minutes and made more than 100 passes? Return the player's surname and team.*

**Hint**: To check if attribute A contains a (sub)string S, use the LIKE keyword (e.g. `A like '%S%'`). The % sign indicates a wildcard.

In [18]:
# Your query goes here
query = """select surname, team
            from {}.world_cup.players as P
            where P.team like '%ia'
            and P.minutes < 200 
            and P.passes > 100""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team
0,Kuzmanovic,Serbia


*2) Find all players who made more than 20 shots. Return all player information in descending order of shots made.*

**Hint**: Sorting results is done via the ORDER BY keyword. The default order is ascending (ASC). If you want descending order, use DESC (e.g. `ORDER BY column_1, column_2 DESC`).

In [19]:
# Your query goes here
query = """select *
            from {}.world_cup.players as P
            where shots > 20
            order by shots desc""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team,position,minutes,shots,passes,tackles,saves
0,Gyan,Ghana,forward,501,27,151,1,0
1,Villa,Spain,forward,529,22,169,2,0
2,Messi,Argentina,forward,450,21,321,10,0


*3) Which team has the highest average number of passes per minute played? Return the team's name and average number of passes per minute.*

**Hint #1**: You can compute a team's average number of passes per minute played by dividing the total number of passes by the total number of minutes. To force floating point division, multiply one operand by 1.0.

**Hint #2**: Consider using the LIMIT keyword.

In [20]:
# Your query goes here
query = """SELECT *, (1.0 * total_passes / total_minutes) AS avg_ppm
            FROM (
                SELECT team, sum(passes) AS total_passes, sum(minutes) AS total_minutes
                FROM {}.world_cup.players_and_teams
                GROUP BY team)
            ORDER BY avg_ppm DESC
            LIMIT 1""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,total_passes,total_minutes,avg_ppm
0,Spain,3701,5940,0.623064


*4) Find the goalkeepers of teams that played more than four games. List the surname of the goalkeeper, the team, and the number of minutes the goalkeeper played.*

**Hint**: Use the `PlayersExt` table.

In [21]:
# Your query goes here
query = """select surname, team, minutes
            FROM {}.world_cup.players_and_teams
            where position like "goal%"
            and games > 4""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team,minutes
0,Casillas,Spain,540
1,Romero,Argentina,450
2,Stekelenburg,Netherlands,540
3,Neuer,Germany,540
4,Julio Cesar,Brazil,450
5,Kingson,Ghana,510
6,Muslera,Uruguay,570
7,Villar,Paraguay,480


*5) How many players on a team with a ranking lower than 10 played more than 350 minutes? Return a single number in a column named "superstar".*

**Hint**: To rename a column, use the AS keyword (e.g. `SELECT column_1 AS label`).

In [22]:
# Your query goes here
query = """select count(surname) as superstar
            FROM {}.world_cup.players_and_teams
            where ranking < 10
            and minutes > 350""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,superstar
0,54


*6) What is the average number of passes made by forwards? What about midfielders? Write one query that returns both values with the corresponding position.*

**Hint**: Use the GROUP BY keyword. GROUP BY statements are often used in conjuction with aggregate functions like AVG(), SUM() or COUNT(). 

In [23]:
# Your query goes here
query = """select position, avg(passes) as avg_passes
            FROM {}.world_cup.players
            where position in ("forward", "midfielder")
            group by position""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,position,avg_passes
0,forward,50.825175
1,midfielder,95.271930


### Advanced Queries
Now, on to more challenging questions...

*1) Find all pairs of teams that have the same number of `goalsFor` as well as the same number of `goalsAgainst` as each other. Return the team pairs and their respective numbers of `goalsFor` and `goalsAgainst` (make sure to return each pair only once!).*

**Hint**: You basically need to do a "self join" of the `Teams` table. For that, you need to join different name aliases of the same table. Check [this page](https://www.w3schools.com/sql/sql_join_self.asp) for help.

In [24]:
# Your query goes here
query = """select *
        FROM {}.world_cup.teams as T1, {}.world_cup.teams as T2
        LIMIT 3""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,team_1,ranking_1,games_1,wins_1,draws_1,losses_1,goalsFor_1,goalsAgainst_1,yellowCards_1,redCards_1
0,Greece,13,3,1,0,2,2,5,5,0,North Korea,105,3,0,0,3,1,12,2,0
1,Greece,13,3,1,0,2,2,5,5,0,Greece,13,3,1,0,2,2,5,5,0
2,Greece,13,3,1,0,2,2,5,5,0,Honduras,38,3,0,1,2,0,3,7,0


In [25]:
query = """select A.team AS team_A, B.team AS team_B, A.goalsFor, A.goalsAgainst
            FROM {}.world_cup.teams as A, {}.world_cup.teams as B
            where A.team < B.team 
            and A.goalsFor = B.goalsFor
            and A.goalsAgainst = B.goalsAgainst""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team_A,team_B,goalsFor,goalsAgainst
0,Cameroon,Greece,2,5
1,England,Nigeria,3,5
2,England,South Africa,3,5
3,Italy,Mexico,4,5
4,Chile,England,3,5
5,Chile,Nigeria,3,5
6,Chile,South Africa,3,5
7,Nigeria,South Africa,3,5
8,Australia,Denmark,3,6


*2) Find all teams with a ranking below 30 where no player has made more than 150 passes. Return the team's name and ranking.*

**Hint #1**: Consider using the HAVING keyword.

**Hint #2**: You may also want to look up nested queries.

In [26]:
# Your query goes here
query = """ SELECT *
            FROM (
                select team, min(ranking) as ranking, max(passes) as max_passes
                from {}.world_cup.players_and_teams
                group by team
            )
            where ranking < 30
            and max_passes <= 150
            """.format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,ranking,max_passes
0,France,9,125
1,Switzerland,24,138
2,Nigeria,21,111


*3) Which team has the highest ratio of goalsFor to goalsAgainst?*

In [27]:
# Your query goes here
query = """select team, goalsFor, goalsAgainst, (1.0 * goalsFor / goalsAgainst) as ratio
            FROM {}.world_cup.teams
            order by ratio desc
            limit 1""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,goalsFor,goalsAgainst,ratio
0,Portugal,7,1,7.0


*4) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [28]:
# Your query goes here
query = """select team, avg(passes) as avg_passes
            FROM {}.world_cup.players_and_teams
            where position like "defender"
            group by team
            having avg_passes > 150
            order by avg_passes desc""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,avg_passes
0,Spain,213.000000
1,Brazil,190.000000
2,Germany,189.833333
3,Netherlands,182.500000
4,Mexico,152.142857


## Exercise 2 - Google Cloud datasets

Let's build some queries for the `accidents_2016` data from the NHTSA traffic facilities dataset, which is publicly available on Google BigQuery.

### Setting up

In [29]:
# Create a reference to the traffic facilities dataset
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
traffic_dataset = client.get_dataset(traffic_ref)

In [30]:
# Construct a reference to the "accident_2016" table
accidents_ref = traffic_ref.table("accident_2016")

# API request - fetch the table
accidents_table = client.get_table(accidents_ref)

In [31]:
# Display schema
accidents_table.schema

[SchemaField('state_number', 'INTEGER', 'NULLABLE', None, 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC). For more info on the codes, please look at <C1/V1/D1/PC1/P1/NM1 State Number> section in the pdf: https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812315', (), None),
 SchemaField('state_name', 'STRING', 'NULLABLE', None, 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC).', (), None),
 SchemaField('consecutive_number', 'INTEGER', 'NULLABLE', None, 'This data element is the unique case number assigned to each crash. It appears on each data file and is used to merge information from the data files together. xxxxxx Two Characters for State Code followed by Four Characters for Case Number', (), None

In [32]:
# Show a preview of the "accident_2016" table
client.list_rows(accidents_table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,16,Idaho,160065,2,1,1,0,1,2,3,...,99,19,Recent Previous Crash Scene Nearby (Since 1989),0,None,0,None,1,0,2016-05-21 18:05:00+00:00
1,24,Maryland,240021,1,1,0,0,0,3,3,...,99,0,None,0,None,0,None,1,0,2016-01-16 14:30:00+00:00
2,31,Nebraska,310185,1,1,0,0,0,2,2,...,99,0,None,0,None,0,None,1,0,2016-12-16 17:10:00+00:00
3,37,North Carolina,370035,2,1,1,0,1,4,5,...,99,0,None,0,None,0,None,1,0,2016-01-22 13:50:00+00:00
4,39,Ohio,391029,1,1,0,0,0,5,5,...,55,0,None,0,None,0,None,2,0,2016-12-17 01:45:00+00:00


### Questions

You can access the table using `from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`.

1) How many crashes happened in each month of 2016?

In [33]:
q3 = """
select month_of_crash, count(*) as count
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by month_of_crash
order by month_of_crash
"""

query_job_3 = client.query(q3)
query_job_3.to_dataframe()

,month_of_crash,count
0,1,2354
1,2,2426
2,3,2694
3,4,2713
4,5,3005
5,6,3025
6,7,3025
7,8,3134
8,9,3154
9,10,3287


2) Let's call the accidents involving more than 5 motor vehichles in transport involved as "massive_crash", and all others as "small crash". Return the number of each type of crash for each weather condition.

**Hint**: Use columns `atmospheric_conditions_1_name` and `number_of_motor_vehicles_in_transport_mvit`.

In [34]:
q4 = """
select
  atmospheric_conditions_1_name,
  (case when number_of_motor_vehicles_in_transport_mvit > 5 then "massive_crash"
  else "small_crash" end) as crash_type,
  count(*) as nb_crashes
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by atmospheric_conditions_1_name, crash_type
order by atmospheric_conditions_1_name
"""

query_job_4 = client.query(q4)
query_job_4.to_dataframe()

,atmospheric_conditions_1_name,crash_type,nb_crashes
0,"Blowing Sand, Soil, Dirt",small_crash,20
1,Blowing Snow,small_crash,10
2,Clear,small_crash,24893
3,Clear,massive_crash,63
4,Cloudy,small_crash,5112
5,Cloudy,massive_crash,14
6,"Fog, Smog, Smoke",small_crash,328
7,"Fog, Smog, Smoke",massive_crash,1
8,Freezing Rain or Drizzle,small_crash,17
9,Not Reported,small_crash,1552


3) Find the top 5 states with the highest number of drunk drivers involved in accidents.

In [35]:
q5 = """
select state_name, sum(number_of_drunk_drivers) as nb_drunk_drivers
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by state_name
order by nb_drunk_drivers desc
limit 5
"""

query_job_5 = client.query(q5)
query_job_5.to_dataframe()

,state_name,nb_drunk_drivers
0,California,1052
1,Texas,918
2,Florida,665
3,North Carolina,399
4,Ohio,365


4) Let's find out the states where using a cell phone is a major cause of accidents. Return the first 5 state names along with the number of crashes where the driver was distracted "while manipulating cellular phone".

**Hint**: You need to join the `distract_2016` and `accident_2016` tables.

In [36]:
q6 = """
select
  count(distract.driver_distracted_by) as nb_accidents,
  accident.state_name
from bigquery-public-data.nhtsa_traffic_fatalities.distract_2016 distract
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on distract.consecutive_number = accident.consecutive_number
where driver_distracted_by = 6
group by state_name
order by nb_accidents desc
limit 5
"""

query_job_6 = client.query(q6)
query_job_6.to_dataframe()

,nb_accidents,state_name
0,18,Texas
1,9,Missouri
2,9,California
3,8,North Carolina
4,8,Georgia


In [37]:
q6_1 = """
with
  join_acc_dist as (
    select distract.driver_distracted_by, accident.state_name
    from bigquery-public-data.nhtsa_traffic_fatalities.distract_2016 distract
    join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
    on distract.consecutive_number = accident.consecutive_number
    )
select count(driver_distracted_by) as nb_accidents, state_name from join_acc_dist
where driver_distracted_by = 6
group by state_name
order by nb_accidents desc
limit 5
"""

query_job_6_1 = client.query(q6_1)
query_job_6_1.to_dataframe()

,nb_accidents,state_name
0,18,Texas
1,9,Missouri
2,9,California
3,8,North Carolina
4,8,Georgia


5) Sometimes, a "live animal" is a reason that a driver maneuvers and causes an accident. In which months of the year do such accidents happen the most? Return the month and the number of accidents. 

**Hint**: You need to join the `maneuver_2016` and `accident_2016` tables. 

In [38]:
q7 = """
select
  accident.month_of_crash,
  count(maneuver.driver_maneuvered_to_avoid) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
group by month_of_crash
order by nb_accidents desc
"""

query_job_7 = client.query(q7)
query_job_7.to_dataframe()

,month_of_crash,nb_accidents
0,10,384579
1,9,369018
2,8,366678
3,11,355797
4,7,353925
5,6,353925
6,5,351585
7,12,338130
8,4,317421
9,3,315198


6) In the state of Alaska, in which months do "poor road conditions" cause the most accidents? Return the month and the number of accidents. 

**Hint**: Again, you will need the `maneuver_2016` table. 

In [39]:
q8 = """
select
  accident.month_of_crash,
  count(maneuver.driver_maneuvered_to_avoid) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
  and accident.state_name = "Alaska"
group by month_of_crash
order by nb_accidents desc
"""

query_job_8 = client.query(q8)
query_job_8.to_dataframe()

,month_of_crash,nb_accidents
0,11,1053
1,7,1053
2,8,936
3,6,936
4,4,819
5,10,819
6,9,702
7,1,702
8,12,702
9,5,702


7) Sometimes, a factor of the accident is that the vehicle has a problem with the "brake system". Filter the accidents with this factor and return the `driver_maneuvered_to_avoid_name` column along with the related number of accidents.
 
**Hint**: You need to join the `maneuver_2016` and `factor_2016` tables.

In [40]:
q9 = """
select
  maneuver.driver_maneuvered_to_avoid_name,
  count(maneuver.consecutive_number) as num_incidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.factor_2016 factor
on maneuver.consecutive_number = factor.consecutive_number
where factor.contributing_circumstances_motor_vehicle = 2
group by driver_maneuvered_to_avoid_name
order by num_incidents desc
"""

query_job_9 = client.query(q9)
query_job_9.to_dataframe()

,driver_maneuvered_to_avoid_name,num_incidents
0,Driver Did Not Maneuver To Avoid,97
1,Not Reported,93
2,Motor Vehicle,25
3,Phantom/Non-Contact Motor Vehicle,5
4,No Driver Present/Unknown if Driver Present,3
5,Live Animal,1


8) Sometimes, a non-motorist is involved in a crash. For each state, find the number of crashes in which an "inattentive" person was involved. Return the state name and number of crashes.

**Hint**: You need to join the `nmcrash_2016` and `accident_2016` tables.

In [41]:
q10 = """
select
  accident.state_name,
  count(nmcrash.consecutive_number) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.nmcrash_2016 nmcrash
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on nmcrash.consecutive_number = accident.consecutive_number
where nmcrash.non_motorist_contributing_circumstances = 6
group by state_name
order by nb_accidents desc
"""

query_job_10 = client.query(q10)
query_job_10.to_dataframe()

,state_name,nb_accidents
0,Louisiana,17
1,Georgia,16
2,North Carolina,14
3,Missouri,12
4,Washington,9
5,California,8
6,Ohio,7
7,Texas,7
8,New Jersey,6
9,Florida,4
